# Sequence-to-sequence LSTM outlier detector deployment

Wrap a keras seq2seq-LSTM python model for use as a prediction microservice in seldon-core and deploy on seldon-core running on Minikube or a Kubernetes cluster using GCP.

## Dependencies

- [helm](https://github.com/helm/helm)
- [minikube](https://github.com/kubernetes/minikube)
- [s2i](https://github.com/openshift/source-to-image) >= 1.1.13

Python packages:
- keras: pip install keras
- tensorflow: https://www.tensorflow.org/install/pip

## Task

The outlier detector needs to spot anomalies in electrocardiograms (ECG's). The dataset "ECG5000" contains 5000 ECG's, originally obtained from [Physionet](https://physionet.org/cgi-bin/atm/ATM) under the name "BIDMC Congestive Heart Failure Database(chfdb)", record "chf07". The data has been pre-processed in 2 steps: first each heartbeat is extracted, and then each beat is made equal length via interpolation. The data is labeled and contains 5 classes. The first class which contains almost 60% of the observations is seen as "normal" while the others are outliers. The seq2seq-LSTM algorithm is trained on some heartbeats from the first class and needs to flag the other classes as anomalies. The plot below shows an example ECG for each of the classes.

![ECGs](images/ecg.png)

## Train locally

Train on some inlier ECG's. The data can be downloaded [here](http://www.timeseriesclassification.com/description.php?Dataset=ECG5000) and should be extracted in the [data](./data) folder.

In [ ]:
!python train.py \
--dataset './data/ECG5000_TEST.arff' \
--data_range 0 2627 \
--minmax \
--timesteps 140 \
--encoder_dim 20 \
--decoder_dim 40 \
--output_activation 'sigmoid' \
--dropout 0 \
--learning_rate 0.005 \
--loss 'mean_squared_error' \
--epochs 100 \
--batch_size 32 \
--validation_split 0.2 \
--print_progress \
--save

The plot below shows a typical prediction (*red line*) of an inlier (class 1) ECG compared to the original (*blue line*) after training the seq2seq-LSTM model.

![inlier_ecg](images/inlier_ecg.png)

On the other hand, the model is not good at fitting ECG's from the other classes, as illustrated in the chart below:

![outlier_ecg](images/outlier_ecg.png)

The predictions in the above charts are made on ECG's the model has not seen before. The differences in scale are due to the sigmoid output layer and do not affect the prediction accuracy.

## Test using Kubernetes cluster on GCP or Minikube

Run the outlier detector as a model or a transformer. If you want to run the anomaly detector as a transformer, change the SERVICE_TYPE variable from MODEL to TRANSFORMER [here](./.s2i/environment), set MODEL = False and change ```OutlierSeq2SeqLSTM.py``` to:

```python
from CoreSeq2SeqLSTM import CoreSeq2SeqLSTM

class OutlierSeq2SeqLSTM(CoreSeq2SeqLSTM):
    """ Outlier detection using a sequence-to-sequence (seq2seq) LSTM model.
    
    Parameters
    ----------
        threshold (float) :  reconstruction error (mse) threshold used to classify outliers
        reservoir_size (int) : number of observations kept in memory using reservoir sampling
    """
    def __init__(self,threshold=0.003,reservoir_size=50000,model_name='seq2seq',load_path='./models/'):
        
        super().__init__(threshold=threshold,reservoir_size=reservoir_size,
                         model_name=model_name,load_path=load_path)
```

In [ ]:
MODEL = True

Pick Kubernetes cluster on GCP or Minikube.

In [ ]:
MINIKUBE = True

In [ ]:
if MINIKUBE:
    !minikube start --memory 4096 
else:
    !gcloud container clusters get-credentials standard-cluster-1 --zone europe-west1-b --project seldon-demos

Create a cluster-wide cluster-admin role assigned to a service account named “default” in the namespace “kube-system”.

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin \
--serviceaccount=kube-system:default

In [ ]:
!kubectl create namespace seldon

Add current context details to the configuration file in the seldon namespace.

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Create tiller service account and give it a cluster-wide cluster-admin role.

In [ ]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

Check deployment rollout status and deploy seldon/spartakus helm charts.

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

In [ ]:
!helm install ../../../helm-charts/seldon-core-controller --name seldon-core --set usage_metrics.enabled=true --namespace seldon-system

Check deployment rollout status for seldon core.

In [ ]:
!kubectl rollout status statefulset.apps/seldon-operator-controller-manager -n seldon-system

Install Ambassador API gateway

In [ ]:
!helm install stable/ambassador --name ambassador --set image.tag=0.40.2

In [ ]:
!kubectl rollout status deployment.apps/ambassador

If Minikube used: create docker image for outlier detector inside Minikube using s2i. Besides the transformer image and the demo specific model image, the general model image for the Seq2Seq LSTM outlier detector is also available from Docker Hub as ***seldonio/outlier-s2s-lstm-model:0.1***.

In [ ]:
if MINIKUBE & MODEL:
    !eval $(minikube docker-env) && \
    s2i build . seldonio/seldon-core-s2i-python3:0.4 seldonio/outlier-s2s-lstm-model-demo:0.1
elif MINIKUBE:
    !eval $(minikube docker-env) && \
    s2i build . seldonio/seldon-core-s2i-python3:0.4 seldonio/outlier-s2s-lstm-transformer:0.1

Install outlier detector helm charts and set *threshold* and *reservoir_size* hyperparameter values.

In [ ]:
if MODEL:
    !helm install ../../../helm-charts/seldon-od-model \
        --name outlier-detector \
        --namespace=seldon \
        --set model.type=seq2seq \
        --set model.seq2seq.image.name=seldonio/outlier-s2s-lstm-model-demo:0.1 \
        --set model.seq2seq.threshold=0.002 \
        --set model.seq2seq.reservoir_size=50000 \
        --set oauth.key=oauth-key \
        --set oauth.secret=oauth-secret \
        --set replicas=1
else:
    !helm install ../../../helm-charts/seldon-od-transformer \
        --name outlier-detector \
        --namespace=seldon \
        --set outlierDetection.enabled=true \
        --set outlierDetection.name=outlier-s2s-lstm \
        --set outlierDetection.type=seq2seq \
        --set outlierDetection.seq2seq.image.name=seldonio/outlier-s2s-lstm-transformer:0.1 \
        --set outlierDetection.seq2seq.threshold=0.002 \
        --set outlierDetection.seq2seq.reservoir_size=50000 \
        --set oauth.key=oauth-key \
        --set oauth.secret=oauth-secret \
        --set model.image.name=seldonio/outlier-s2s-lstm-model:0.1

## Port forward Ambassador

Run command in terminal:

```
kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

## Import rest requests, load data and test requests

In [ ]:
import numpy as np
from utils import get_payload, rest_request_ambassador, send_feedback_rest, ecg_data

ecg_data, ecg_labels = ecg_data(dataset='TRAIN')
X = ecg_data[0,:].reshape(1,ecg_data.shape[1],1)
label = ecg_labels[0].reshape(1)
print(X.shape)
print(label.shape)

Test the rest requests with the generated data. It is important that the order of requests is respected. First we make predictions, then we get the "true" labels back using the feedback request. If we do not respect the order and eg keep making predictions without getting the feedback for each prediction, there will be a mismatch between the predicted and "true" labels. This will result in errors in the produced metrics.

In [ ]:
request = get_payload(X)

In [ ]:
response = rest_request_ambassador("outlier-detector","seldon",request,endpoint="localhost:8003")

If the outlier detector is used as a transformer, the output of the anomaly detection is added as part of the metadata. If it is used as a model, we send model feedback to retrieve custom performance metrics.

In [ ]:
if MODEL:
    send_feedback_rest("outlier-detector","seldon",request,response,0,label,endpoint="localhost:8003")

## Analytics

Install the helm charts for prometheus and the grafana dashboard

In [ ]:
!helm install ../../../helm-charts/seldon-core-analytics --name seldon-core-analytics \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false \
    --namespace seldon

## Port forward Grafana dashboard

Run command in terminal:

```
kubectl port-forward $(kubectl get pods -n seldon -l app=grafana-prom-server -o jsonpath='{.items[0].metadata.name}') -n seldon 3000:3000
```

You can then view an analytics dashboard inside the cluster at http://localhost:3000/dashboard/db/prediction-analytics?refresh=5s&orgId=1. Your IP address may be different. get it via minikube ip. Login with:

Username : admin

password : password (as set when starting seldon-core-analytics above)

Import the outlier-detector-s2s-lstm dashboard from ../../../helm-charts/seldon-core-analytics/files/grafana/configs.

## Run simulation

- Sample random ECG from dataset.
- Get payload for the observation.
- Make a prediction.
- Send the "true" label with the feedback if the detector is run as a model.

It is important that the prediction-feedback order is maintained. Otherwise there will be a mismatch between the predicted and "true" labels.

View the progress on the grafana "Outlier Detection" dashboard. Most metrics need the outlier detector to be run as a model since they need model feedback.

In [ ]:
import time
n_requests = 100
n_samples, sample_length = ecg_data.shape
for i in range(n_requests):
    idx = np.random.choice(n_samples)
    X = ecg_data[idx,:].reshape(1,sample_length,1)
    label = ecg_labels[idx].reshape(1)
    request = get_payload(X)
    response = rest_request_ambassador("outlier-detector","seldon",request,endpoint="localhost:8003")
    if MODEL:
        send_feedback_rest("outlier-detector","seldon",request,response,0,label,endpoint="localhost:8003")
    time.sleep(1)

In [ ]:
if MINIKUBE:
    !minikube delete